# Natural Language Processing - Assignment 3, Detecting fraudulent email

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd /content/gdrive/Shareddrives/NLP_AT3/archive/Phishing_Email.csv

Import neccessary packages

In [ ]:
!pip install -q -U "tensorflow-text==2.13.*"
!pip install -q "tf-model-official==2.13.*"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("/shareddrive/NLP AT3/achieve/phishing_emails.csv")
data.head()

Preprocessing

- Figure out how BERT model works
- Preprocessing step required

In [ ]:
data.isnull().sum()
data.dropna(inplace=True) # Remove null values
data.drop_duplicates() # Remove duplicate values

In [ ]:
print(df['Email Type'].value_counts()) # print Email Type value

In [ ]:
from sklearn.preprocessing import LabelEncoder

#encode labels
label_encoder = LabelEncoder()
data['email_type'] = label_encoder.fit_transform(data['Email Type'])

In [ ]:
#train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
# x_train , x_test , y_train , y_test = train_test_split(df, test_size=0.2, random_state=42)

train_texts, temp_texts, train_labels, temp_labels = train_test_split(data['email_text'], data['Email Type'], test_size=0.3, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(temp_texts['email_text'], temp_labels['Email Type'], test_size=0.5, random_state=42)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

Model

Results

# Working Attempt

Note: Set runtime to T4 GPU otherwise training on CPU will take 81 hours rather than 1 hour

In [ ]:
!pip install accelerate -U
# pip install transformers[torch]

In [ ]:
import pandas as pd

data = pd.read_csv('/content/Phishing_Email.csv')

In [ ]:
data.sample()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18650 entries, 0 to 18649
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  18650 non-null  int64 
 1   Email Text  18634 non-null  object
 2   Email Type  18650 non-null  object
dtypes: int64(1), object(2)
memory usage: 437.2+ KB


In [ ]:
from sklearn.model_selection import train_test_split
import re

#function to clean text, and prepare for tokenisation
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'\W+', '', text)
    text = text.strip()
    return text

In [ ]:
data['email_text'] = data['Email Text'].apply(clean_text)

In [ ]:
print(data['email_text'].isnull().sum())

0


In [ ]:
from sklearn.preprocessing import LabelEncoder

#encode labels
label_encoder = LabelEncoder()
data['email_type'] = label_encoder.fit_transform(data['Email Type'])

In [ ]:
# Split the data into training and temp (remaining) data
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    data['email_text'], data['email_type'], test_size=0.2, random_state=42)  # 20% for temp (validation + test)

# Split the temp data into validation and test sets
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch

class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Convert to Dataset
train_dataset = EmailDataset(train_encodings, train_labels.tolist())
val_dataset = EmailDataset(val_encodings, val_labels.tolist())

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(data['Email Type'])))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
histpry = trainer.train()
# history

Step,Training Loss
10,0.681300
20,0.671500
30,0.662600
40,0.664400
50,0.669100
60,0.691800
70,0.668400
80,0.663400
90,0.668400
100,0.636700


TrainOutput(global_step=5595, training_loss=0.6573929725745834, metrics={'train_runtime': 4033.3614, 'train_samples_per_second': 11.097, 'train_steps_per_second': 1.387, 'total_flos': 1.17768508379136e+16, 'train_loss': 0.6573929725745834, 'epoch': 3.0})

In [ ]:
torch.save(trainer)

In [ ]:
# Tokenize the test data
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512)

# Convert the test labels to integers if not already
#test_labels = label_encoder.transform(test_labels)  # Assuming you've used LabelEncoder previously

# Create a Dataset object for test data
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = EmailDataset(test_encodings)

In [ ]:
# Evaluate the model on the test set
test_result = trainer.evaluate(test_dataset)
print(test_result)

KeyError: 1

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

# Evaluate the model on validation dataset
predictions = trainer.predict(eval_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(eval_labels, pred_labels)
print("Accuracy:", accuracy)

# Calculate confusion matrix
conf_matrix = confusion_matrix(eval_labels, pred_labels)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
import matplotlib.pyplot as plt

# Access the training loss and validation loss from the Trainer object
train_loss = trainer.state.log_history['train_loss']
eval_loss = trainer.state.log_history['eval_loss']

# Plot training and validation loss
plt.plot(train_loss, label='Training Loss')
plt.plot(eval_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Access the training and validation accuracies from the Trainer object
train_metrics = trainer.state.log_history['train_metrics']
eval_metrics = trainer.state.log_history['eval_metrics']

train_accuracy = [metrics['accuracy'] for metrics in train_metrics]
eval_accuracy = [metrics['eval_accuracy'] for metrics in eval_metrics]

# Plot training and validation accuracy
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(eval_accuracy, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Access the training loss and accuracy from the Trainer object
train_metrics = trainer.state.log_history['train_metrics']

train_loss = [metrics['loss'] for metrics in train_metrics]
train_accuracy = [metrics['accuracy'] for metrics in train_metrics]

# Plot training loss
plt.plot(train_loss, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()

# Plot training accuracy
plt.plot(train_accuracy, label='Training Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()
plt.show()

In [ ]:
# Example new text
new_texts = ["Y	FINANCIAL INVESTMENT CENTER INSIDER NEWSÂ See bottom of page for removal instructions. Stock Market has you concerned? Join the flight to quality. Make over 25% annually, paid monthly by Factoring Account Receivables.Â  Discover what banks have known for decades. Harness the power and liquidity of Factoring ï¿½ Collateralized Account Receivables.Â  A safe haven to make strong returns, while stock markets are volatile.Â  Please Be over the age of 18. Serious inquiries only.Â  For A Free In-Depth Information Package. Please fill out the form below.CONTACT ME First Name:Last Name:Address:City:State:Zip Number:Country:Home Number:Business Number:Email Address:Best time to call:Â To Be Removed: Email Address:",
             "retention plan for four managers todd , let ' s work up a plan tomorrow for jeff gossett ' s managers before the completion of your survey . if your schedule does not allow it please recommend someone in comp . who can help or please provide guidance for alternatives . thanks for your help . hgm - - - - - - - - - - - - - - - - - - - - - - forwarded by hector mcloughlin / corp / enron on 10 / 10 / 2000 06 : 02 pm - - - - - - - - - - - - - - - - - - - - - - - - - - - from : sally beck @ ect 10 / 10 / 2000 05 : 57 pm to : hector mcloughlin / corp / enron @ enron cc : subject : retention plan for four managers hector : the key employees that jeff has identified are senior specialists , with the exception of will kelly . if you remember , these are the risk employees that i was concerned about when steve jackson tendered his letter of resignation . steve will be taking a similar role with a competitor , and it is not inconceivable that he might approach some enron employees to join him and help ensure his success with his new employer . we don ' t want this to happen . waiting until the end of november to implement an effective retention plan does not make sense . please call me tomorrow and let ' s talk about how we can jump start this . thanks . - - sally - - - - - - - - - - - - - - - - - - - - - - forwarded by sally beck / hou / ect on 10 / 10 / 2000 05 : 39 pm - - - - - - - - - - - - - - - - - - - - - - - - - - - enron north america corp . from : jeffrey c gossett 10 / 10 / 2000 05 : 38 pm to : sally beck / hou / ect @ ect cc : subject : retention plan for four managers - - - - - - - - - - - - - - - - - - - - - - forwarded by jeffrey c gossett / hou / ect on 10 / 10 / 2000 05 : 38 pm - - - - - - - - - - - - - - - - - - - - - - - - - - - enron north america corp . from : hector mcloughlin @ enron 10 / 09 / 2000 10 : 26 am to : jeffrey c gossett / hou / ect @ ect , todd burke / na / enron @ enron cc : subject : retention plan for four managers jeff , i spoke with todd burke regarding a retention strategy for the managers we talked about . he told me that he would make a recommendation after he completes the pay practices survery by mid - november in order to accurately compensate them . he is concerned that we may present an inaccurate plan for retention without market data . if this is a very urgent issue for you please let us know . if you suspect that the managers are being made offers then please let us know . hgm"]

# Tokenize the new data
new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=512)

# Convert to a Dataset object (no labels this time)
class PredictDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

new_dataset = PredictDataset(new_encodings)

# Predict
predictions = trainer.predict(new_dataset)

# Convert predictions to labels
predicted_labels = label_encoder.inverse_transform(predictions.predictions.argmax(-1))
print(predicted_labels)

['Safe Email' 'Safe Email']
